### CNN
* http://andrew.gibiansky.com/blog/machine-learning/convolutional-neural-networks/
* http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
* https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721
* https://algotravelling.com/ru/%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5-%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5-%D1%8D%D1%82%D0%BE-%D0%B2%D0%B5%D1%81%D0%B5%D0%BB%D0%BE-3/
* https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/
* https://medium.com/@karpathy/yes-you-should-understand-backprop-e2f06eab496b
* https://www.asozykin.ru/courses/nnpython
* https://www.tensorflow.org/tutorials/deep_cnn#convolutional-neural-networks

In [2]:
from scipy import ndimage
import numpy as np
import pandas as pd
import os
from IPython.display import display, Image
from scipy import misc
import matplotlib.pyplot as plt
import tensorflow
import seaborn

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

%matplotlib inline

Using TensorFlow backend.


### Keras

Виктор
* арима для каждого игрока
* последние 2 месяца
* фб профит либа

* уменьшить число фич - можно побить бейслайн

vgg для 224\224
keras vgg
уменьшить количество каналов в 4 раза
resnet: первый слов свертка 7х7 пулинг 3х3 убрать
конва + идентити блок




In [1]:
img_width = img_height = 32
img_channels = 3

# VGG16

In [3]:
first = 32
second = 64
third = 128
fourth = 256
filter_size = (3,3)
initializer = 'orthogonal'
padding = 'same'
activation = 'relu'
dropout = 0.5

model = Sequential()
model.add(Conv2D(first, filter_size, padding=padding, kernel_initializer=initializer, 
           input_shape=(img_width, img_height, img_channels)))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Conv2D(first, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))


model.add(Conv2D(second, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Conv2D(second, filter_size, padding=padding', kernel_initializer=initializer, use_bias=False))
model.add(BatchNormalization())
model.add(Activation(activation))


model.add(Conv2D(third, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Conv2D(third, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Conv2D(third, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(Conv2D(fourth, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Conv2D(fourth, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Conv2D(fourth, filter_size, padding=padding, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))

model.add(GlobalAveragePooling2D())
model.add(Dense(1028, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(512, kernel_initializer=initializer))
model.add(BatchNormalization())
model.add(Activation(activation))
model.add(Dense(256, kernel_initializer=initializer))
model.add(Activation(activation))

# https://arxiv.org/pdf/1412.6806.pdf 
# STRIVING FOR SIMPLICITY

In [2]:
model.count_params()

NameError: name 'model' is not defined

In [4]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [6]:
batch_size = 128

dir_transformed = '../output/transformed32'
dir_train = os.path.join(dir_transformed, 'train_bal')
dir_val = os.path.join(dir_transformed, 'validation_bal')
dir_test = os.path.join(dir_transformed, 'test')

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    dir_train,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(
    dir_val,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
    dir_test,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=False)

Found 19540 images belonging to 256 classes.
Found 2560 images belonging to 256 classes.
Found 7680 images belonging to 1 classes.


In [ ]:
callback_1 = TensorBoard(log_dir='./logs/logs', histogram_freq=0, write_graph=False, write_images=False)
callback_2 = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
callback_3 = ModelCheckpoint('model.hdf5', monitor='val_acc', save_best_only=True, verbose=0)

In [ ]:
%%time
model.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=20,
    validation_data=val_generator,
    validation_steps=20,
    callbacks=[callback_1, callback_2, callback_3]
)

In [ ]:
model.summary()

In [ ]:
results = model.predict_generator(test_generator, steps=300)
results = np.argmax(results, axis=1)+1

In [ ]:
# create submission dataframe (in specified kaggle format)
sub = pd.DataFrame(
    data=[row for row in zip([i.split("/")[1] for i in test_generator.filenames], results.astype(int).tolist())],
    columns=['image', 'class']
)

In [ ]:
sub.to_csv('keras_do-05.csv', encoding='utf-8', index=False)